<a href="https://colab.research.google.com/github/TekyAms/Jeu_pierre_feuille_ciseaux/blob/main/Copie_de_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Charger les données
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

train_df.head()

In [ ]:
# Convertir les étiquettes en nombres
train_df['label'] = train_df['label'].map({'ham': 0, 'spam': 1})
test_df['label'] = test_df['label'].map({'ham': 0, 'spam': 1})

# Diviser les données en features et labels
X_train = train_df['message'].values
y_train = train_df['label'].values
X_test = test_df['message'].values
y_test = test_df['label'].values


In [ ]:
# Tokenizer pour convertir les textes en séquences
tokenizer = keras.preprocessing.text.Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Padding pour uniformiser la longueur des séquences
X_train_pad = keras.preprocessing.sequence.pad_sequences(X_train_seq, maxlen=120, padding='post', truncating='post')
X_test_pad = keras.preprocessing.sequence.pad_sequences(X_test_seq, maxlen=120, padding='post', truncating='post')

# Créer le modèle
model = keras.Sequential([
    keras.layers.Embedding(input_dim=10000, output_dim=16, input_length=120),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compiler le modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
history = model.fit(X_train_pad, y_train, epochs=10, validation_data=(X_test_pad, y_test))


In [ ]:
def predict_message(message):
    # Prétraiter le message
    seq = tokenizer.texts_to_sequences([message])
    pad_seq = keras.preprocessing.sequence.pad_sequences(seq, maxlen=120, padding='post', truncating='post')
    # Prédire la probabilité
    prob = model.predict(pad_seq)[0][0]
    # Déterminer le label
    label = 'spam' if prob > 0.5 else 'ham'
    # Retourner la probabilité et le label
    return [prob, label]

# Tester la fonction
print(predict_message("Congratulations! You've won a $1,000 Walmart gift card. Go to http://bit.ly/123456 to claim now."))
print(predict_message("Hey, are we still meeting for lunch today?"))


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
